In [1]:
import warnings
warnings.filterwarnings("ignore")
## setting up the language model
import os
from dotenv import load_dotenv
from langchain_together import ChatTogether
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain import LLMChain

## load the environment variables
load_dotenv()

True

In [1]:
import langchain
langchain.__version__

'0.3.19'

In [2]:
llm = ChatTogether(api_key=os.getenv("TOGETHER_AI_API_KEY"),temperature=0.0, model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")

In [3]:
from langchain_community.document_loaders import PyPDFLoader


# Load PDF
loaders = [
    PyPDFLoader("./Gautam_Naik_DSML_CV.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [4]:
len(docs)

2

In [5]:
docs[0].page_content

'Gautam\n \nNaik\n \nMachine\n \nLearning\n \nEngineer\n \n&\n \nData\n \nScientist\n \nGoa-India,\n  \n+91\n \n9421873737,\n  \ng a u t a m n a i k 1 9 9 4 @ g m a i l . c o m\n,\n \nl i n k e d i n . c o m / i n / g a u t a m n a i k 1 9 9 4\n,\n  \ng a u t a m n a i k . c o m\n,\n  \ng i t h u b . c o m / g a u t a m n a i k 1 9 9 4\n \n \n \n●\n \nMachine\n \nLearning\n \nEngineer\n \nwith\n \n9\n \nyears\n \nof\n \nsoftware\n \nengineering\n \nexperience\n \nand\n \n4\n \nyears\n \nof\n \nhands-on\n \ndata\n \nscience\n \n&\n \nML\n \nexpertise\n.\n \nSkilled\n \nin\n \nPython,\n \nSQL,\n \nand\n \ncloud-based\n \nML\n \ndeployment,\n \nwith\n \na\n \nstrong\n \nfoundation\n \nin\n \nend-to-end\n \nmodel\n \ndevelopment\n \nand\n \nMLOps.\n  \n \n●\n \nBuilt\n \nAI-driven\n \nforecasting\n \nmodels\n \nthat\n \nimproved\n \nbusiness\n \ndecisions\n \nby\n \n15%\n \nand\n \nan\n \nautomated\n \ntrading\n \nsystem\n \nusing\n \nAWS\n \nLambda\n \n&\n \nCDK.\n \nExperienced\n \nin\n 

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
    separators=["\n\n","\n","."," "] 
)

In [7]:
splits = text_splitter.split_documents(docs)

In [8]:
len(splits)

24

In [9]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()

In [10]:
from langchain_community.vectorstores import FAISS, InMemoryVectorStore

db = FAISS.from_documents(splits, embedding)

print(db.index.ntotal)

24


In [13]:
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=HuggingFaceEmbeddings()
)

In [11]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever()
)

In [ ]:
query = """
We are looking for a machine learning engineer with experience in building and deploying machine learning models.
The candidate should have a background in Python, Generative AI, time series forecasting and cloud platforms like AWS or Azure.
"""

In [13]:
result = qa_chain({"query": query})

/var/folders/jf/1r7jb1_s6ys1p0n0n56s2qsh0000gn/T/ipykernel_2232/3834357911.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


In [14]:
print(result["result"])

Based on the provided context, it seems that Gautam Naik, a Machine Learning Engineer & Data Scientist, meets most of the requirements you specified. Here's a breakdown of his qualifications:

1. **Experience in building and deploying machine learning models**: Gautam has 4 years of hands-on data science & ML expertise, which includes building AI-driven forecasting models that improved business decisions by 15%. He also built an automated trading system using AWS Lambda & CDK.

2. **Strong background in Python**: Gautam is skilled in Python, which is one of the primary programming languages used in machine learning.

3. **Experience with cloud platforms**: Gautam has experience with AWS, specifically with AWS Lambda, CDK, and DynamoDB. He also has a strong foundation in cloud-based ML deployment.

4. **Problem-solving skills**: As a machine learning engineer, Gautam has likely developed strong problem-solving skills, which are essential for working with complex machine learning models.

In [25]:
db.save_local("cv_db")

In [20]:
new_db = FAISS.load_local("cv_db", embedding, allow_dangerous_deserialization=True)